## AlexNet
2010年开始举办的[ILSVRC](http://image-net.org/challenges/LSVRC/)比赛基于一个百万量级的图片数据集, 提出一个图像1000分类的挑战. 前两年在比赛中脱颖而出的都是经过人工挑选特征, 再通过`SVM`或者`随机森林`这样在过去十几年中非常成熟的机器学习方法进行分类的算法. 

在2012年, 由 [Alex Krizhevsky](https://www.cs.toronto.edu/~kriz/), [Ilya Sutskever](http://www.cs.toronto.edu/~ilya/), [Geoffrey Hinton](http://www.cs.toronto.edu/~hinton/)提出了一种使用卷积神经网络的方法, 以 [0.85](http://image-net.org/challenges/LSVRC/2012/results.html#abstract) 的`top-5`正确率一举获得当年分类比赛的冠军, 超越使用传统方法的第二名10个百分点, 震惊了当时的学术界, 从此开启了人工智能领域的新篇章.

### `CIFAR10`

这是一个包含60000张$32\times32$图片的数据库, 包含50000张训练集和10000张测试集, 这里我们提供一个脚本帮助我们读取数据

In [1]:
import tensorflow as tf
from utils import cifar10_input

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# 我们定义一个批次有64个样本
batch_size = 64

In [7]:
# 先下载数据
#链接：https://pan.baidu.com/s/1fbThmv8LDhLRemA62m3nlQ 
#提取码：j1d8 

# 获取训练集
# 在使用随机梯度下降法的时候, 训练集要求打乱样本
data_dir = 'cifar10_data'

train_imgs, train_labels = cifar10_input.inputs(eval_data=False, 
                                                data_dir='./cifar10_data/cifar-10-batches-bin/', 
                                                batch_size=batch_size, 
                                                shuffle=True)

# 获取测试集
# 测试集不需要打乱样本
val_imgs, val_labels = cifar10_input.inputs(eval_data=True, 
                                            data_dir='./cifar10_data/cifar-10-batches-bin/', 
                                            batch_size=batch_size, 
                                            shuffle=False)

In [8]:
train_examples = cifar10_input.NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN # 训练样本的个数
val_examples = cifar10_input.NUM_EXAMPLES_PER_EPOCH_FOR_EVAL       # 测试样本的个数

In [9]:
# 像之前一样, 我们构造几个生成变量的函数
def variable_weight(shape, stddev=5e-2):
    init = tf.truncated_normal_initializer(stddev=stddev)
    return tf.get_variable(shape=shape, initializer=init, name='weight')

def variable_bias(shape):
    init = tf.constant_initializer(0.1)
    return tf.get_variable(shape=shape, initializer=init, name='bias')

In [10]:
def conv(x, ksize, out_depth, strides, padding='SAME', act=tf.nn.relu, scope='conv_layer', reuse=None):
    """构造一个卷积层
    Args:
        x: 输入
        ksize: 卷积核的大小, 一个长度为2的`list`, 例如[3, 3]
        output_depth: 卷积核的个数
        strides: 卷积核移动的步长, 一个长度为2的`list`, 例如[2, 2]
        padding: 卷积核的补0策略
        act: 完成卷积后的激活函数, 默认是`tf.nn.relu`
        scope: 这一层的名称(可选)
        reuse: 是否复用
    
    Return:
        out: 卷积层的结果
    """
    # 这里默认数据是NHWC输入的
    in_depth = x.get_shape().as_list()[-1]
    
    with tf.variable_scope(scope, reuse=reuse):
        # 先构造卷积核
        shape = ksize + [in_depth, out_depth]
        with tf.variable_scope('kernel'):
            kernel = variable_weight(shape)
            
        strides = [1, strides[0], strides[1], 1]
        # 生成卷积
        conv = tf.nn.conv2d(x, kernel, strides, padding, name='conv')
        
        # 构造偏置
        with tf.variable_scope('bias'):
            bias = variable_bias([out_depth])
            
        # 和偏置相加
        preact = tf.nn.bias_add(conv, bias)
        
        # 添加激活层
        out = act(preact)
        
        return out

In [11]:
def max_pool(x, ksize, strides, padding='SAME', name='pool_layer'):
    """构造一个最大值池化层
    Args:
        x: 输入
        ksize: pooling核的大小, 一个长度为2的`list`, 例如[3, 3]
        strides: pooling核移动的步长, 一个长度为2的`list`, 例如[2, 2]
        padding: pooling的补0策略
        name: 这一层的名称(可选)
    
    Return:
        pooling层的结果
    """
    return tf.nn.max_pool(x, [1, ksize[0], ksize[1], 1], [1, strides[0], strides[1], 1], padding, name=name)

In [12]:
def fc(x, out_depth, act=tf.nn.relu, scope='fully_connect', reuse=None):
    """构造一个全连接层
    Args:
        x: 输入
        out_depth: 输出向量的维数
        act: 激活函数, 默认是`tf.nn.relu`
        scope: 名称域, 默认是`fully_connect`
        reuse: 是否需要重用
    """
    in_depth = x.get_shape().as_list()[-1]
    
    # 构造全连接层的参数
    with tf.variable_scope(scope, reuse=reuse):
        # 构造权重
        with tf.variable_scope('weight'):
            weight = variable_weight([in_depth, out_depth])
            
        # 构造偏置项
        with tf.variable_scope('bias'):
            bias = variable_bias([out_depth])
        
        # 一个线性函数
        fc = tf.nn.bias_add(tf.matmul(x, weight), bias, name='fc')
        
        # 激活函数作用
        out = act(fc)
        
        return out

In [13]:
def alexnet(inputs, reuse=None):
    """构建 Alexnet 的前向传播
    Args:
        inpus: 输入
        reuse: 是否需要重用
        
    Return:
        net: alexnet的结果
    """
    # 首先我们声明一个变量域`AlexNet`
    with tf.variable_scope('AlexNet', reuse=reuse):
        # 第一层是 5x5 的卷积, 卷积核的个数是64, 步长是 1x1, padding是`VALID`
        net = conv(inputs, [5, 5], 64, [1, 1], padding='VALID', scope='conv1')
        
        # 第二层是 3x3 的池化, 步长是 2x2, padding是`VALID`
        net = max_pool(net, [3, 3], [2, 2], padding='VALID', name='pool1')
        
        # 第三层是 5x5 的卷积, 卷积核的个数是64, 步长是 1x1, padding是`VALID`
        net = conv(net, [5, 5], 64, [1, 1], scope='conv2')
        
        # 第四层是 3x3 的池化, 步长是 2x2, padding是`VALID`
        net = max_pool(net, [3, 3], [2, 2], padding='VALID', name='pool2')
        
        # 将矩阵拉长成向量
        net = tf.reshape(net, [-1, 6*6*64])
        
        # 第五层是全连接层, 输出个数为384
        net = fc(net, 384, scope='fc3')
        
        # 第六层是全连接层, 输出个数为192
        net = fc(net, 192, scope='fc4')
        
        # 第七层是全连接层, 输出个数为10, 注意这里不要使用激活函数
        net = fc(net, 10, scope='fc5', act=tf.identity)
        
        return net

通过 alexnet 构建训练和测试的输出

In [14]:
train_out = alexnet(train_imgs)

注意当再次调用 alexnet 函数时, 如果要使用之前调用时产生的变量值, **必须**要重用变量域

In [15]:
val_out = alexnet(val_imgs, reuse=True)

#### 定义损失函数
这里真实的 labels 不是一个 one_hot 型的向量, 而是一个数值, 因此我们使用 

In [16]:
with tf.variable_scope('loss'):
    train_loss = tf.losses.sparse_softmax_cross_entropy(labels=train_labels, logits=train_out, scope='train')
    val_loss = tf.losses.sparse_softmax_cross_entropy(labels=val_labels, logits=val_out, scope='val')

#### 定义正确率`op`

In [17]:
with tf.name_scope('accuracy'):
    with tf.name_scope('train'):
        train_acc = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(train_out, axis=-1, output_type=tf.int32), train_labels), tf.float32))
    with tf.name_scope('train'):
        val_acc = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(val_out, axis=-1, output_type=tf.int32), val_labels), tf.float32))

#### 构造训练`op`

In [18]:
lr = 0.01

opt = tf.train.MomentumOptimizer(lr, momentum=0.9)
train_op = opt.minimize(train_loss)

In [19]:
from utils.learning import train

In [20]:
train(train_op, train_loss, train_acc, val_loss, val_acc, 20000, batch_size)

[train]: step 0 loss = nan acc = 0.0000 (0.0083 / batch)
[val]: step 0 loss = nan acc = 0.0156
[train]: step 1000 loss = nan acc = 0.0000 (0.0212 / batch)
[train]: step 2000 loss = nan acc = 0.0000 (0.0222 / batch)
[train]: step 3000 loss = nan acc = 0.0000 (0.0223 / batch)
[train]: step 4000 loss = nan acc = 0.0000 (0.0224 / batch)
[val]: step 4000 loss = nan acc = 0.0000
[train]: step 5000 loss = nan acc = 0.0000 (0.0222 / batch)
[train]: step 6000 loss = nan acc = 0.0000 (0.0224 / batch)
[train]: step 7000 loss = nan acc = 0.0000 (0.0223 / batch)
[train]: step 8000 loss = nan acc = 0.0000 (0.0224 / batch)
[val]: step 8000 loss = nan acc = 0.0000
[train]: step 9000 loss = nan acc = 0.0000 (0.0215 / batch)
[train]: step 10000 loss = nan acc = 0.0000 (0.0218 / batch)
[train]: step 11000 loss = nan acc = 0.0000 (0.0225 / batch)
[train]: step 12000 loss = nan acc = 0.0000 (0.0228 / batch)
[val]: step 12000 loss = nan acc = 0.0000
[train]: step 13000 loss = nan acc = 0.0312 (0.0227 / batc

可以看到, 20000步的训练后, `AlexNet`在训练集和测试集分别达到了`0.97`和`0.72`的准确率